# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [38]:
import kagglehub
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from tqdm import tqdm





In [39]:
# Download the latest version of the dataset
path = kagglehub.dataset_download("rishabhkausish/reddit-depression-dataset")

# Verifique o conteúdo da pasta para localizar o nome do arquivo
print("Path to dataset files:", path)
print("Files in the directory:", os.listdir(path))  # Lista os arquivos no diretório baixado

# Use o nome exato do arquivo CSV dentro do diretório para carregar no pandas
csv_file = os.path.join(path, "reddit_depression_dataset.csv")  # Substitua pelo nome exato do arquivo
df_original = pd.read_csv(csv_file)
df = df_original



Path to dataset files: /Users/pedrotomielo/.cache/kagglehub/datasets/rishabhkausish/reddit-depression-dataset/versions/1
Files in the directory: ['reddit_depression_dataset.csv']


/var/folders/bf/j88nff6527n1q_zj02k2gf8r0000gn/T/ipykernel_67096/2194657197.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv_file)


In [40]:
df = df[['title','body','upvotes','created_utc','num_comments','label']]
display(df)
df['created_utc'] = pd.to_datetime(df['created_utc'],unit='s')
df['created_utc'].dt.tz_localize('UTC').dt.tz_convert('Europe/Amsterdam')
df

,title,body,upvotes,created_utc,num_comments,label
0,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,NaN,0.0
1,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0
2,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0
3,"""I want to be like water. I want to slip throu...",NaN,25.0,1.416512e+09,2.0,0.0
4,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0
...,...,...,...,...,...,...
2470773,Alone as usual,I just argued with my mom an hour ago; she alw...,9.0,1.672529e+09,6.0,1.0
2470774,Help needed urgently!,OK basically non of my friend's or family invi...,5.0,1.672530e+09,7.0,1.0
2470775,I know what your thinking,Don't do It. Not today. Its gonna be ok if you...,10.0,1.672530e+09,8.0,1.0
2470776,how do I stop having such a cynical view on ma...,"So I don't know, I've recently realized that t...",8.0,1.672531e+09,9.0,1.0


/var/folders/bf/j88nff6527n1q_zj02k2gf8r0000gn/T/ipykernel_67096/1994711439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_utc'] = pd.to_datetime(df['created_utc'],unit='s')


,title,body,upvotes,created_utc,num_comments,label
0,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,2014-07-14 03:35:09,NaN,0.0
1,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,2014-09-13 00:31:19,1.0,0.0
2,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,2014-11-20 04:31:58,1.0,0.0
3,"""I want to be like water. I want to slip throu...",NaN,25.0,2014-11-20 19:38:05,2.0,0.0
4,Who am I?,You could take any one cell in my body and kil...,5.0,2014-11-20 20:36:52,4.0,0.0
...,...,...,...,...,...,...
2470773,Alone as usual,I just argued with my mom an hour ago; she alw...,9.0,2022-12-31 23:30:10,6.0,1.0
2470774,Help needed urgently!,OK basically non of my friend's or family invi...,5.0,2022-12-31 23:39:16,7.0,1.0
2470775,I know what your thinking,Don't do It. Not today. Its gonna be ok if you...,10.0,2022-12-31 23:41:28,8.0,1.0
2470776,how do I stop having such a cynical view on ma...,"So I don't know, I've recently realized that t...",8.0,2022-12-31 23:56:57,9.0,1.0


In [41]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2470778 entries, 0 to 2470777
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   title         object        
 1   body          object        
 2   upvotes       float64       
 3   created_utc   datetime64[ns]
 4   num_comments  float64       
 5   label         float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 113.1+ MB


,upvotes,created_utc,num_comments,label
count,2.470714e+06,2470672,2.356801e+06,2.470672e+06
mean,6.260360e+01,2019-08-22 11:43:59.377216,1.526266e+01,1.944455e-01
min,0.000000e+00,2008-01-26 02:17:12,-3.000000e+00,0.000000e+00
25%,5.000000e+00,2019-03-10 08:01:44.750000128,4.000000e+00,0.000000e+00
50%,7.000000e+00,2020-01-07 04:23:28,7.000000e+00,0.000000e+00
75%,1.000000e+01,2021-01-25 02:04:42.249999872,1.400000e+01,0.000000e+00
max,1.288660e+05,2022-12-31 23:58:35,2.113100e+04,1.000000e+00
std,9.474559e+02,NaN,7.867238e+01,3.957733e-01


In [42]:

# Define os valores NaN em 'num_comments' e 'upvotes' para 0
df.loc[df[['num_comments', 'upvotes']].isnull().any(axis=1), ['num_comments', 'upvotes']] = 0
df.isnull().sum()

title               23
body            461051
upvotes              0
created_utc        106
num_comments         0
label              106
dtype: int64

In [43]:
df

,title,body,upvotes,created_utc,num_comments,label
0,Deep thoughts underdog,"Only when we start considering ourselves, the ...",0.0,2014-07-14 03:35:09,0.0,0.0
1,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,2014-09-13 00:31:19,1.0,0.0
2,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,2014-11-20 04:31:58,1.0,0.0
3,"""I want to be like water. I want to slip throu...",NaN,25.0,2014-11-20 19:38:05,2.0,0.0
4,Who am I?,You could take any one cell in my body and kil...,5.0,2014-11-20 20:36:52,4.0,0.0
...,...,...,...,...,...,...
2470773,Alone as usual,I just argued with my mom an hour ago; she alw...,9.0,2022-12-31 23:30:10,6.0,1.0
2470774,Help needed urgently!,OK basically non of my friend's or family invi...,5.0,2022-12-31 23:39:16,7.0,1.0
2470775,I know what your thinking,Don't do It. Not today. Its gonna be ok if you...,10.0,2022-12-31 23:41:28,8.0,1.0
2470776,how do I stop having such a cynical view on ma...,"So I don't know, I've recently realized that t...",8.0,2022-12-31 23:56:57,9.0,1.0


# Limpar texto

In [44]:
import re

def limpar_texto(texto):
    # Verifica se o valor é uma string
    if isinstance(texto, str):
        # 1. Remover links (URLs)
        texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
        
        # 2. Remover números
        texto = re.sub(r'\d+', '', texto)
        
        # 3. Remover caracteres especiais 
        texto = re.sub(r'[^\w\s]', '', texto)
        
        # 4. Remover emojis
        emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # Emoticons
            u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
            u"\U0001F680-\U0001F6FF"  # Transporte e símbolos de mapa
            u"\U0001F1E0-\U0001F1FF"  # Bandeiras
            u"\U00002700-\U000027BF"  # Diversos símbolos e pictogramas
            u"\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE)
        texto = emoji_pattern.sub(r'', texto)
        
        # 5. Remover espaços extras
        texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto if isinstance(texto, str) else ""



# Ativar o tqdm para o pandas
tqdm.pandas()


df.loc[:, 'title'] = df['title'].progress_apply(limpar_texto)
df.loc[:, 'body'] = df['body'].progress_apply(limpar_texto)





100%|██████████| 2470778/2470778 [00:11<00:00, 221821.09it/s]

 33%|███▎      | 812883/2470778 [00:12<00:25, 64597.28it/s]


KeyboardInterrupt: 

In [ ]:
df

### 1. Stopwords
Stopwords são palavras muito comuns que aparecem frequentemente em textos, mas que geralmente não carregam informações significativas para tarefas de análise de texto, como sentimentos, classificação, etc. Exemplos de stopwords incluem palavras como "a", "de", "o", "para", "é", "em", entre outras. Embora elas sejam importantes para a gramática e o entendimento básico da frase, muitas vezes elas são removidas no pré-processamento porque não ajudam no entendimento do significado principal do texto.

In [ ]:
# Baixe o conjunto de stop words em português (execute uma vez)
nltk.download('stopwords')

# Defina as stop words em português
stop_words = set(stopwords.words('english'))

# Função para remover stop words
def remover_stopwords(texto):
    palavras = texto.split()
    palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in stop_words]
    return ' '.join(palavras_filtradas)

### 2. Stemming
Stemming é o processo de reduzir palavras às suas "raízes", removendo sufixos e prefixos. A ideia é que palavras com significados semelhantes podem ser reduzidas à mesma forma base. Por exemplo, palavras como "jogando", "jogou" e "jogar" podem ser reduzidas à raiz comum "jog".



In [ ]:
# Inicializar o stemmer
stemmer = PorterStemmer()

# Função para aplicar stemming
def aplicar_stemming(texto):
    palavras = texto.split()
    palavras_stem = [stemmer.stem(palavra) for palavra in palavras]
    return ' '.join(palavras_stem)


### 3. Lemmatization
Lemmatization também é um processo de redução de palavras à sua forma base, mas, ao contrário do stemming, a lematização considera o contexto gramatical e semanticamente mais correto. A lematização transforma uma palavra em sua forma de dicionário (ou "lemma")

In [ ]:
import spacy

# Carregar o modelo de inglês do spacy
nlp = spacy.load("en_core_web_sm")

# Função para aplicar lematização
def aplicar_lemmatization(texto):
    doc = nlp(texto)
    palavras_lemmatizadas = [token.lemma_ for token in doc]
    return ' '.join(palavras_lemmatizadas)


# Aplicar funçoes

In [ ]:
def preprocessamento_completo(texto):
    # Remover stop words
    texto = remover_stopwords(texto)
    
    #texto = aplicar_stemming(texto)
    

    texto = aplicar_lemmatization(texto) 
    
    return texto


In [ ]:
df['title'] = df['title'].progress_apply(preprocessamento_completo)
df['body'] = df['body'].progress_apply(preprocessamento_completo)


## Separando em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

# Suponha que você tenha features (X) e labels (y)
X_train, X_test, y_train, y_test = train_test_split(df['body'], df['label'], test_size=0.2, random_state=42)



# TESTES 

# 
sentences (X_train no seu caso):

Este é o conjunto de dados de treinamento, que deve ser uma lista de listas de palavras. Cada documento ou frase é representado como uma lista de tokens (palavras). No exemplo, X_train deve conter frases já tokenizadas.
vector_size (ou size em algumas versões do gensim):

Especifica o número de dimensões dos vetores de palavras, ou seja, o tamanho do vetor de embedding. Um valor comum é 100, mas esse número pode variar dependendo da complexidade e tamanho do seu vocabulário.
window:

Define o tamanho da janela de contexto, ou seja, quantas palavras à esquerda e à direita de uma palavra-alvo o modelo deve considerar para entender o contexto. Um valor típico é 5, mas pode variar. Janela maior captura contexto mais amplo.
min_count:

Estabelece o número mínimo de ocorrências de uma palavra para que ela seja considerada no treinamento. Se definido como 1, todas as palavras do conjunto de dados são incluídas. Esse valor pode ser ajustado para excluir palavras muito raras que podem introduzir ruído.
workers:

Define o número de threads que o modelo deve usar para o treinamento. 4 threads indica que o modelo treina em paralelo, o que pode acelerar o processo em máquinas com múltiplos núcleos.

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(X_train, vector_size=100, window=5, min_count=1, workers=4)



In [ ]:
display(model)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Treina o classificador
classificador = LogisticRegression()
classificador.fit(model, y_train)

# Faz previsões
y_pred = classificador.predict(model)

# Avalia o desempenho
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))